In [1]:
using Random
using DataFrames
using CSV

In [2]:
function build_lattice_df(Lx,Ly)
    
    N = Lx*Ly
    interactions = DataFrame(["$spin" => [] for spin in 1:N])    
    for spin in 1:N
        up = mod(spin-1-Lx,N)
        down = mod(spin-1+Lx,N)
        right = Int(((spin-1)-((spin-1)%Lx)) + mod(((spin-1)%Lx)+1,Lx))
        left = Int(((spin-1)-((spin-1)%Lx)) + mod(((spin-1)%Lx)-1,Lx))
        push!(interactions[!,"$spin"],[up+1,down+1,left+1,right+1])
    end
    
    return interactions
end

build_lattice_df (generic function with 1 method)

In [3]:
function Calculate_Ising_Energy(config,interactions,J)
    
    N = length(config)
    energy = 0
    
    for spin in (1:N)
        neighbors = interactions[!,"$spin"][1,1]
        energy_contributions = 0.5*-1*J*config[spin].*config[neighbors]
        energy += sum(energy_contributions)
    end
    
    return energy
    
end

Calculate_Ising_Energy (generic function with 1 method)

In [4]:
function build_deltaE_df(beta)
    
    possible_sum_neighbors = collect(-4:2:4)
    spins = [1,-1]
    deltaEs = DataFrame(["$spin,$poss" => [] for poss in possible_sum_neighbors for spin in spins])    
    for sum_neighbors in possible_sum_neighbors
        spinup_deltaE = 2*sum_neighbors
        spindown_deltaE = -2*sum_neighbors 
        push!(deltaEs[!,"1,$sum_neighbors"],spinup_deltaE)
        push!(deltaEs[!,"-1,$sum_neighbors"],spindown_deltaE)
    end
    
    return deltaEs
end

build_deltaE_df (generic function with 1 method)

In [5]:
function Metropolis_update_function(config,energy,interactions,beta,J,deltaE_vals)
    
    rand_spin_flip_index = rand(1:length(config),1)[1]
    new_config = copy(config)

    spin = Int(config[rand_spin_flip_index])
    neighbors = interactions[!,"$rand_spin_flip_index"][1,1]
    sum_neighbors = Int(sum(config[neighbors]))
    deltaE = deltaE_vals[!,"$spin,$sum_neighbors"][1]
    exp_beta_deltaE = exp(-1*beta*deltaE)
    rand_value = rand(Float64,1)[1]
    
    if exp_beta_deltaE > rand_value
        new_config[rand_spin_flip_index] = -1*config[rand_spin_flip_index]
    else
        deltaE = 0.
    end
    
    return new_config, deltaE
end

Metropolis_update_function (generic function with 1 method)

In [6]:
function Ising_cluster_update(config,energy,interactions,beta,J,energy_func)
    # angle defining flipping axis
    N = length(config)
    cluster = []
    spins_to_check = []
    initial_spin = rand(1:N)
    prob = 1-exp(-(2*beta))
    append!(cluster, initial_spin)
    append!(spins_to_check, initial_spin)
    while !isempty(spins_to_check)
        # check all neighbours of spins_to_check
        site = spins_to_check[1]
        spin_val = config[site]
        neighbors = interactions[!,"$site"][1,1]
        neighbors_not_in_cluster = setdiff(neighbors,cluster)
        for i in neighbors_not_in_cluster
            spin_neighbor = config[i]
            if spin_neighbor == spin_val
                if prob > rand()
                    append!(cluster, i)
                    append!(spins_to_check, i)
                end
            end
        end
        #remove the one we just checked
        popfirst!(spins_to_check)
    end
    
    for i in cluster
        config[i] = -1*config[i]
    end
    
    new_energy = energy_func(config,interactions,J)
    deltaE = new_energy-energy
    
    return config, deltaE
end

Ising_cluster_update (generic function with 1 method)

In [7]:
function MonteCarlo_Ising(beta,J,Lx,Ly,warmup_steps,steps,steps_per_step,build_lattice_function,build_update_probs,update_function,energy_function)
    
    # system information
    N = Lx*Ly
    interactions = build_lattice_function(Lx,Ly)
    deltaEs = build_deltaE_df(beta)
    initial_config = rand((-1,1),N)
    
    # initialize dataframes
    fulldf = DataFrame(:Es => Float64[],:avE => Float64[])
    maindf = DataFrame(:avenergy => Float64[],:avenergy2 => Float64[],:rawMs => Float64[],:avM => Float64[],:avM2 => Float64[])
    
    # warmup steps - track Es and Ms but not in averages
    config = initial_config
    energy = energy_function(config,interactions,J)
    M = 0.
    fullsumE = 0
    for step in (1:warmup_steps)
        for inner_steps in (1:steps_per_step)
            new_config, deltaE = update_function(config,energy,interactions,beta,J,energy_function)
#             new_config,deltaE = update_function(config,energy,interactions,beta,J,deltaEs)
            config = copy(new_config)
            new_energy = energy+deltaE
            energy = new_energy
            fullsumE = fullsumE+energy
            M = sum(new_config)
        end
        push!(fulldf,(energy,fullsumE/(step*steps_per_step)))
    end
    
    # begin MC steps and observable tracking
    sumE = 0
    sumE2 = 0
    sumM = 0
    sumM2 = 0
    for step in (1:steps)
        for inner_step in (1:steps_per_step)
            new_config,deltaE = update_function(config,energy,interactions,beta,J,energy_function)
#             new_config,deltaE = update_function(config,energy,interactions,beta,J,deltaEs)
            config = copy(new_config)
            new_energy = energy+deltaE
            energy = new_energy
            fullsumE = fullsumE+energy
        end
        M = sum(config)
        push!(fulldf,(energy,fullsumE/((step*steps_per_step)+warmup_steps)))
        absM = abs(M)
        sumE = sumE+energy
        sumE2 = sumE2+energy^2
        sumM = sumM+absM
        sumM2 = sumM2+absM^2
        push!(maindf, (sumE/step,sumE2/step,M,sumM/step,sumM2/step))
    end
        
    return maindf,fulldf
end

MonteCarlo_Ising (generic function with 1 method)

# Q 1 a

In [65]:
Ts = [1.0,1/3]
Ls = [16]
J = 1
N_chains = 5
n_steps = 1000000
warmup = Int(n_steps/10)

for L in Ls
    println("L = ",L)
    for T in Ts
        beta = 1/T
        println("T = ",T," and beta = ",beta)
        T = round(T,digits=3)
        for chain in 1:N_chains
            println("chain #",chain)
            @time begin
            avgs,full = MonteCarlo_Ising(beta,J,L,L,warmup,n_steps,1,build_lattice_df,build_deltaE_df,Metropolis_update_function,Calculate_Ising_Energy);
            end
            CSV.write("./data/q1a/L_$(L)/avgvals_T_$(T)_chain$(chain).csv",  avgs, writeheader=true)
            CSV.write("./data/q1a/L_$(L)/fullavgvals_T_$(T)_chain$(chain).csv",  full, writeheader=true)
        end
    end
end

L = 16
T = 1.0 and beta = 1.0
chain #1
  5.567951 seconds (65.80 M allocations: 6.095 GiB, 24.23% gc time, 3.08% compilation time)
chain #2
  5.426347 seconds (65.29 M allocations: 6.068 GiB, 28.49% gc time)
chain #3
  5.166985 seconds (65.29 M allocations: 6.068 GiB, 24.45% gc time)
chain #4
  5.219140 seconds (65.29 M allocations: 6.091 GiB, 27.74% gc time)
chain #5
  5.229418 seconds (65.29 M allocations: 6.068 GiB, 25.30% gc time)
T = 0.3333333333333333 and beta = 3.0
chain #1
  5.863231 seconds (65.29 M allocations: 6.068 GiB, 24.97% gc time)
chain #2
  5.589845 seconds (65.29 M allocations: 6.068 GiB, 24.76% gc time)
chain #3
  5.568546 seconds (65.29 M allocations: 6.068 GiB, 26.46% gc time)
chain #4
  5.503721 seconds (65.29 M allocations: 6.068 GiB, 25.45% gc time)
chain #5
  5.585944 seconds (65.29 M allocations: 6.068 GiB, 28.19% gc time)


In [46]:
Ts = [1.0,1/3]
Ls = [16]
J = 1
N_chains = 5
n_steps = 1000000
warmup = Int(n_steps/10)

for L in Ls
    println("L = ",L)
    for T in Ts
        beta = 1/T
        println("T = ",T," and beta = ",beta)
        T = round(T,digits=3)
        for chain in 2:N_chains
            println("chain #",chain)
            @time begin
            avgs,full = MonteCarlo_Ising(beta,J,L,L,warmup,n_steps,(L^2)/2,build_lattice_df,build_deltaE_df,Metropolis_update_function,Calculate_Ising_Energy);
            end
            CSV.write("./data/q1a/L_$(L)/avgvals2_T_$(T)_chain$(chain).csv",  avgs, writeheader=true)
            CSV.write("./data/q1a/L_$(L)/allvals2_T_$(T)_chain$(chain).csv",  full, writeheader=true)
        end
    end
end

L = 16
T = 1.0 and beta = 1.0
chain #2
Warmup steps over!
451.069405 seconds (3.14 G allocations: 676.394 GiB, 33.61% gc time)
chain #3
Warmup steps over!
477.814737 seconds (3.14 G allocations: 676.394 GiB, 33.53% gc time)
chain #4
Warmup steps over!
480.002901 seconds (3.14 G allocations: 676.394 GiB, 33.50% gc time)
chain #5
Warmup steps over!
487.751320 seconds (3.14 G allocations: 676.394 GiB, 33.18% gc time)
T = 0.3333333333333333 and beta = 3.0
chain #2
Warmup steps over!
488.955939 seconds (3.14 G allocations: 676.396 GiB, 32.73% gc time)
chain #3
Warmup steps over!
489.598386 seconds (3.14 G allocations: 676.386 GiB, 32.68% gc time)
chain #4
Warmup steps over!
513.325932 seconds (3.14 G allocations: 676.396 GiB, 32.48% gc time)
chain #5
Warmup steps over!
518.629061 seconds (3.14 G allocations: 676.396 GiB, 32.12% gc time)


# Q 1 b, c, d

Need to continue runs for L = 32

In [63]:
Ts = 2.0:0.05:4.0
Ls = [32]
J = 1
N_chains = 10
n_steps = 5000
warmup = Int(n_steps/5)

for L in Ls
    println("L = ",L)
    for T in Ts
        beta = 1/T
        println("T = ",T," and beta = ",beta)
        T = round(T,digits=2)
        for chain in 1:N_chains
            println("chain #",chain)
            avgs,full = MonteCarlo_Ising(beta,J,L,L,warmup,n_steps,1,build_lattice_df,build_deltaE_df,Ising_cluster_update,Calculate_Ising_Energy);
#             CSV.write("./data/q1b/L_$(L)/allvals_T_$(T)_chain$(chain).csv",  full, writeheader=true)
            CSV.write("./data/q1b_cluster/L_$(L)/avgvals_T_$(T)_chain$(chain).csv",  avgs, writeheader=true)
        end
    end
end

L = 4
T = 1.0 and beta = 1.0
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.05 and beta = 0.9523809523809523
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.1 and beta = 0.9090909090909091
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.15 and beta = 0.8695652173913044
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.2 and beta = 0.8333333333333334
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.25 and beta = 0.8
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.3 and beta = 0.7692307692307692
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.35 and beta = 0.7407407407407407
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #

chain #10
T = 1.15 and beta = 0.8695652173913044
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.2 and beta = 0.8333333333333334
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.25 and beta = 0.8
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.3 and beta = 0.7692307692307692
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.35 and beta = 0.7407407407407407
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.4 and beta = 0.7142857142857143
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.45 and beta = 0.6896551724137931
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.5 and beta = 0.6666666666666666
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain 

chain #9
chain #10
T = 1.3 and beta = 0.7692307692307692
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.35 and beta = 0.7407407407407407
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.4 and beta = 0.7142857142857143
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.45 and beta = 0.6896551724137931
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.5 and beta = 0.6666666666666666
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.55 and beta = 0.6451612903225806
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.6 and beta = 0.625
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.65 and beta = 0.6060606060606061
chain #1
chain #2
chain #3
chain #4
chain #5
chai

chain #8
chain #9
chain #10
T = 1.45 and beta = 0.6896551724137931
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.5 and beta = 0.6666666666666666
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.55 and beta = 0.6451612903225806
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.6 and beta = 0.625
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.65 and beta = 0.6060606060606061
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.7 and beta = 0.5882352941176471
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.75 and beta = 0.5714285714285714
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.8 and beta = 0.5555555555555556
chain #1
chain #2
chain #3
chain #4
chai

chain #7
chain #8
chain #9
chain #10
T = 1.6 and beta = 0.625
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.65 and beta = 0.6060606060606061
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.7 and beta = 0.5882352941176471
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.75 and beta = 0.5714285714285714
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.8 and beta = 0.5555555555555556
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.85 and beta = 0.5405405405405405
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.9 and beta = 0.5263157894736842
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.95 and beta = 0.5128205128205129
chain #1
chain #2
chain #3
chai

chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.75 and beta = 0.5714285714285714
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.8 and beta = 0.5555555555555556
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.85 and beta = 0.5405405405405405
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.9 and beta = 0.5263157894736842
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.95 and beta = 0.5128205128205129
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.0 and beta = 0.5
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.05 and beta = 0.48780487804878053
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.1 and beta = 0.47619047619047616
chain #1
chain #2
chai

chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.9 and beta = 0.5263157894736842
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.95 and beta = 0.5128205128205129
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.0 and beta = 0.5
chain #1
chain #2


LoadError: InterruptException:

Finer T grid for smaller system sizes

Need to finish for L = 24!!!! Run L = 32??

In [10]:
Ts = 1.56:0.01:4.0
Ls = [24]
J = 1
N_chains = 10
n_steps = 5000
warmup = Int(n_steps/5)

for L in Ls
    println("L = ",L)
    for T in Ts
        beta = 1/T
        println("T = ",T," and beta = ",beta)
        T = round(T,digits=2)
        for chain in 1:N_chains
            println("chain #",chain)
            avgs,full = MonteCarlo_Ising(beta,J,L,L,warmup,n_steps,1,build_lattice_df,build_deltaE_df,Ising_cluster_update,Calculate_Ising_Energy);
#             CSV.write("./data/q1b/L_$(L)/allvals_T_$(T)_chain$(chain).csv",  full, writeheader=true)
            CSV.write("./data/q1b_cluster/L_$(L)/avgvals_T_$(T)_chain$(chain).csv",  avgs, writeheader=true)
        end
    end
end

L = 24
T = 1.56 and beta = 0.641025641025641
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.57 and beta = 0.6369426751592356
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.58 and beta = 0.6329113924050632
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.59 and beta = 0.6289308176100629
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.6 and beta = 0.625
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.61 and beta = 0.6211180124223602
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.62 and beta = 0.6172839506172839
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 1.63 and beta = 0.6134969325153374
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain

chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.2 and beta = 0.45454545454545453
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.21 and beta = 0.45248868778280543
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.22 and beta = 0.4504504504504504
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.23 and beta = 0.4484304932735426
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.24 and beta = 0.4464285714285714
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.25 and beta = 0.4444444444444444
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.26 and beta = 0.4424778761061947
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.2

chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.83 and beta = 0.35335689045936397
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.84 and beta = 0.35211267605633806
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.85 and beta = 0.3508771929824561
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.86 and beta = 0.3496503496503497
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.87 and beta = 0.3484320557491289
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.88 and beta = 0.3472222222222222
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.89 and beta = 0.3460207612456747
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 2.9 and beta = 0.344

chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 3.46 and beta = 0.28901734104046245
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 3.47 and beta = 0.2881844380403458
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 3.48 and beta = 0.28735632183908044
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 3.49 and beta = 0.28653295128939826
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 3.5 and beta = 0.2857142857142857
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 3.51 and beta = 0.2849002849002849
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 3.52 and beta = 0.2840909090909091
chain #1
chain #2
chain #3
chain #4
chain #5
chain #6
chain #7
chain #8
chain #9
chain #10
T = 3.53 and be